In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# 간단하게 처리하는 방법
from tensorflow.keras.datasets import imdb

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

In [ ]:
print(X_train[1])
print(y_train[1])

In [ ]:
# 문장 길이를 맞춰준다
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
print(X_train[1])

In [ ]:
# 레이어들을 쌓을 모델을 생성
model = Sequential()
# 단어를 임베딩하는데, 5000개의 단어를 120차원으로 내보내겠다
model.add(Embedding(5000, 120))
# RNN - simpleRNN / LSTM
model.add(LSTM(120))
# 긍정/부정을 판단하니까 이진 분류 -> sigmoid 함수 사용
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# 혹시 5회 이상 검증데이터 loss가 증가하면, 과적합될 수 있으므로 학습을 조기종료!
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# 훈련을 거듭하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
# 긍정/부정을 판단하니까 손실함수는 이진 교차 엔트로피, 최적화는 adam, 평가 기준은 acc (출력할때 뜬다)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64, callbacks=[early_stop, model_check])

In [ ]:
# 정확도 측정
print(model.evaluate(X_test, y_test))